In [1]:
import requests
import json
from pandas import DataFrame
import os


In [2]:
urlFmt = "https://dapi.kakao.com/v2/search/image?query={query}&page={page}&size={size}"

In [3]:
query = "봄패션"
page = 1
size = 80
key = "899a1eedbcb6713b4e78b75f1e3ed026"
userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"

In [4]:
session = requests.Session()
session.headers.update({
"Authorization": "KakaoAK %s" % key
})

In [5]:
isEnd = False
page = 1
count = 0
while not isEnd:
    url = urlFmt.format(query=query, page=page, size=size)
    r = session.get(url)
    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
    r.encoding = "utf-8"
    data = json.loads(r.text)
    searchList = data['documents']
    isEnd = bool(data['meta']['is_end'])
    page += 1
    if os.path.exists(query) == False:
        os.mkdir(query)
    for i, v in enumerate(searchList):
        count += 1
        docUrl = v['doc_url']
        imageUrl = v['image_url']
        session.headers.update({
            "Authorization": "KakaoAK %s" % key,
            "Referer": docUrl,
            "User-Agent": userAgent
            })
    if r.status_code != 200:
        continue # 안되면 무시하고 다시 위로
    fileName = "%s/%d05d.png" %(query, count)

    with open(fileName, 'wb') as f:
        r.write(r.raw.read())
print("종료")